In [ ]:
import os
import boto3
from langchain_aws import BedrockEmbeddings
from langchain_community.chat_models import BedrockChat
from langchain_aws import ChatBedrock

def config_env_variables():
    os.environ['AWS_CONFIG_FILE'] = r'C:\Users\223149195\.aws\credentials'
    os.environ['REQUESTS_CA_BUNDLE'] = r'C:\Users\223149195\cacert.pem'  # Set the location of the cacert
    # Turn off proxy if the request hangs
    os.environ['HTTPS_PROXY'] = "http://PITC-Zscaler-ASPAC-Bangalore3PR.proxy.corporate.ge.com:80"
    os.environ['AWS_DEFAULT_PROFILE'] = 'mfa'

config_env_variables()

llm = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs=dict(temperature=0.5),
    credentials_profile_name="mfa"
)

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_histories = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in message_histories:
        message_histories[session_id] = ChatMessageHistory()
    return message_histories[session_id]

In [12]:
from langchain_experimental.sql.base import SQLDatabase

sqlite_db_uri = "sqlite:///publish_layer_1.db"
# When initializing your database
db = SQLDatabase.from_uri(
    sqlite_db_uri
    
)

In [13]:
def get_schema(_):
    return db.get_table_info()

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from prompts import template


prompt_with_history = ChatPromptTemplate.from_messages([
    ("system", template),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

inner_chain = (
    RunnablePassthrough.assign(schema=get_schema) | 
    prompt_with_history | 
    llm | 
    StrOutputParser()
)

sql_chain_with_history = RunnableWithMessageHistory(
    inner_chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history"
)

session_id = "user_session_1"

def query_sql_with_history(question, session_id="default"):
    return sql_chain_with_history.invoke(
        {"question": question},
        config={"configurable": {"session_id": session_id}}
    )

answer = query_sql_with_history("do you have the table schema?", session_id)
print(answer)

KeyError: "Input to ChatPromptTemplate is missing variables {'current_date'}.  Expected: ['current_date', 'history', 'question', 'schema'] Received: ['question', 'history', 'schema']\nNote: if you intended {current_date} to be part of the string and not a variable, please escape it with double curly braces like: '{{current_date}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [1]:
print("hi")

hi


In [1]:
from test_sql import message_histories

print(message_histories)

2025-03-12 12:41:07.069 Thread 'Thread-23': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-12 12:41:07.073 Thread 'Thread-23': missing ScriptRunContext! This warning can be ignored when running in bare mode.


ImportError: cannot import name 'message_histories' from 'test_sql' (c:\Users\223149195\Documents\curiosity_new\test_sql.py)